In [ ]:
from math import pi
import time
import os
import numpy as np
os.environ['IPYCANVAS_DISABLE_OFFSCREEN_CANVAS'] = '0'

from ipycanvas.compat import Canvas
from ipycanvas.call_repeated import set_render_loop

In [ ]:
canvas = Canvas(width=1600, height=1200, layout=dict(width="75%"))
await canvas.display()

class Animate():
    def __init__(self, canvas):
        self.canvas = canvas
        self.canvas.font = '30px Arial'
        self.canvas.text_align = 'center'
        self.canvas.text_baseline = 'middle'
        
        # linear gradient
        color_stops = [
            (0, 'red'),
            (0.2, 'orange'),
            (0.4, 'yellow'),
            (0.6, 'green'),
            (0.8, 'blue'),
            (1, 'purple')
        ]
        self.background_gradient = self.canvas.create_linear_gradient(0, 0, 0, canvas.height,
                                                           color_stops=color_stops)

        # face gradient
        color_stops = [
            (0, '#fca503'),
            (1, '#fcdb03')
        ]
        self.face_gradient = self.canvas.create_radial_gradient(
            canvas.width / 2.0, canvas.height / 2.0, 0,
            canvas.width / 2.0, canvas.height / 2.0, 500,
            color_stops=color_stops
        )

        self.iter = 0
        self.t0 = None

        self.rot = 0

    def __call__(self, dt):
        canvas = self.canvas


        # draw gradient background
        canvas.save()
        canvas.fill_style = self.background_gradient
        canvas.fill_rect(0, 0, canvas.width, canvas.height)
        canvas.restore()

        
        self.iter += 1



        canvas.save()
        canvas.translate(canvas.width/2, canvas.height/2)
        canvas.rotate(self.rot)
        self.rot += dt
        canvas.translate(-canvas.width/2, -canvas.height/2)
    
  
        canvas.fill_style = self.face_gradient
        canvas.fill_circle(canvas.width / 2.0, canvas.height / 2.0, 500)

        canvas.stroke_style = "black"
        canvas.line_width = 30
        canvas.stroke_circle(canvas.width / 2.0, canvas.height / 2.0, 500) # left eye

        canvas.fill_style = "black"
        canvas.fill_circle(canvas.width / 2.7, canvas.height / 3.0, 100)  # Right eye
        
        canvas.stroke_arc(canvas.width / 2.0, canvas.height / 2.0, 400, 0, pi, False)  # Mouth
        canvas.stroke_arc(canvas.width - canvas.width / 2.7, canvas.height / 2.7, 100, 0, pi, True)  
        canvas.restore()



        
        
        if self.iter == 1:
            self.t0 = time.time()
        else:
            t = time.time()
            av_dt = (t - self.t0)/ self.iter
            av_fps = 1.0/av_dt
        

            canvas.fill_text(
                f"Average FPS: {av_fps:.2f}",
                canvas.width / 2.0,
                canvas.height/ 2.0,
            )



set_render_loop(canvas, Animate(canvas))

In [ ]:
from math import pi
from ipycanvas.compat import Canvas

pattern_source = Canvas(width=50, height=50)
await pattern_source.display()
pattern_source.fill_style = "#fec"
pattern_source.fill_rect(0, 0, 50, 50)
pattern_source.stroke_arc(0, 0, 50, 0, 0.5 * pi)

In [ ]:
canvas = Canvas(width=200, height=200)
await canvas.display()
pattern = canvas.create_pattern(pattern_source)
canvas.fill_style = pattern
canvas.fill_rect(0, 0, canvas.width, canvas.height)


In [ ]:
canvas = Canvas(width=400, height=300)
await canvas.display()
canvas.fill_style = "green"
canvas.filter = "blur(1px) contrast(1.4) drop-shadow(-9px 9px 3px #e81)"
canvas.font = "48px serif"
canvas.fill_text("Hello world!", 20, 150)

In [ ]:
from ipywidgets import Image

canvas = Canvas(width=300, height=300)
await canvas.display()


sprite1 = Image.from_file("sprites/smoke_texture0.png")
sprite2 = Image.from_file("sprites/smoke_texture1.png")

canvas.fill_style = "#a9cafc"
canvas.fill_rect(0, 0, 300, 300)

canvas.draw_image(sprite1, 50, 50)
canvas.draw_image(sprite2, 100, 100)

In [ ]:

x = np.linspace(-1, 1, 600)
y = np.linspace(-1, 1, 600)

x_grid, y_grid = np.meshgrid(x, y)

blue_channel = np.array(np.sin(x_grid**2 + y_grid**2) * 255, dtype=np.int32)
red_channel = np.zeros_like(blue_channel) + 200
green_channel = np.zeros_like(blue_channel) + 50

image_data = np.stack((red_channel, blue_channel, green_channel), axis=2)

canvas = Canvas(width=image_data.shape[0], height=image_data.shape[1])
await canvas.display()
canvas.put_image_data(image_data, 0, 0)

In [ ]:
# batch api
shape = (1000,500)
canvas = Canvas(width=shape[0], height=shape[1], layout=dict(width="100%"))
await canvas.display()

n = 100

# random colors
rgb = np.random.randint(0, 255, size=(n, 3), dtype=np.uint8)

# random positions
x = np.random.randint(0, shape[0], size=n)
y = np.random.randint(0, shape[1], size=n)

# random radius
r = np.random.randint(5, 50, size=n)
canvas.fill_style = 'white'
canvas.fill_rect(0,0, canvas.width, canvas.height)
canvas.fill_styled_circles(x=x,y=y, radius=r,color=rgb)
canvas.line_width = 2
canvas.stroke_style = "black"
canvas.stroke_circles( x=x,y=y,radius=r)

In [ ]:
from collections import deque
canvas = Canvas(width=1000, height=800, layout=dict(width="auto"))
await canvas.display()
import time
import itertools


class Animation(object):

    def __init__(self,canvas):

        self.canvas = canvas
        self.canvas.line_cap = 'round'
        self.max_age = 2.0
        self.traces = deque(maxlen=2)
        self.current_trace = []
        self.trace_id = 0
        self.is_down = False
        self.mouse_position = (0, 0)

        self.canvas.on_mouse_move(self.on_mouse_move)
        self.canvas.on_mouse_down(self.on_mouse_down)
        self.canvas.on_mouse_up(self.on_mouse_up)
        self.canvas.on_mouse_out(self.on_mouse_out)

    def add_trace(self, x,y):
        self.current_trace.append((x,y,time.time(), self.trace_id))

    def on_mouse_move(self, x, y):
        # print(x,y)
        self.mouse_position = (x, y)
        if self.is_down:
            self.add_trace(x, y)

    def on_mouse_down(self, x, y):
        self.is_down = True
        self.mouse_position = (x, y)

    def next_trace(self):
        self.traces.append(self.current_trace)
        self.current_trace = []
        self.trace_id += 1

    
    def on_mouse_up(self, x, y):
        self.is_down = False
        self.mouse_position = (x, y)
        self.next_trace()
    
    def on_mouse_out(self, x, y):
        self.is_down = False
        self.mouse_position = (x, y)
        self.next_trace()

    def __call__(self, dt):
        try:
            self.update(dt)
        except Exception as e:
            print(e)
    def update(self, dt):
        self.canvas.clear()
        for trace in itertools.chain([self.current_trace],self.traces):

            if len(trace) == 0:
                continue

            youngest = trace[-1]
            t = youngest[2]
            age = time.time() - t
            if age >= self.max_age:
                continue

           
            last = None
            for i,(x, y,t, tid) in enumerate(trace):

                if i == 0:
                    last = (x, y, t, tid)
                    continue
           
                    


                age = time.time() - t
                if age >= self.max_age:
                    continue
                rel_age = 1.0 - (age / self.max_age)        
                
                # modulate the color
                alpha = rel_age 
                r = int(255 * rel_age)
                g = int(255 - 255 * rel_age)
                b = int(255 * rel_age/2)
                canvas.stroke_style = f'rgba({r}, {g}, {b}, {1})'

                # modulate the line with
                canvas.line_width = (rel_age * 5)**3
                canvas.begin_path()
                canvas.move_to(last[0], last[1])
                canvas.line_to(x, y)
                canvas.stroke()

                last = (x, y, t, tid)

        


set_render_loop(canvas, Animation(canvas=canvas))

In [ ]:
# batch api circles
shape = (1000,500)
canvas = Canvas(width=shape[0], height=shape[1], layout=dict(width="100%"))
await canvas.display()

n = 100

# random colors
rgb = np.random.randint(0, 255, size=(n, 3), dtype=np.uint8)

# random positions
x = np.random.randint(0, shape[0], size=n)
y = np.random.randint(0, shape[1], size=n)

# random width and height
w = np.random.randint(5, 50, size=n)
h = np.random.randint(5, 50, size=n)
canvas.fill_style = 'white'
canvas.fill_styled_rects(x=x, y=y, width=w, height=h, color=rgb)
canvas.line_width = 2
canvas.stroke_style = "black"
canvas.stroke_circles( x=x,y=y,radius=r)

In [ ]:
# batch api rects
shape = (1000,500)
canvas = Canvas(width=shape[0], height=shape[1], layout=dict(width="100%"))
await canvas.display()

n = 100

# random colors
rgb = np.random.randint(0, 255, size=(n, 3), dtype=np.uint8)

# random positions
x = np.random.randint(0, shape[0], size=n)
y = np.random.randint(0, shape[1], size=n)

# linear gradient
color_stops = [
    (0,  "rgb(255,255,255)"),
    (1,  "rgb(0,0,0)"),
]
grad = canvas.create_linear_gradient(0, 0, 0,canvas.height,color_stops=color_stops)


# random width and height
w = np.random.randint(5, 50, size=n)
h = np.random.randint(5, 50, size=n)

# not styled rects a bit larger
canvas.fill_style = grad
canvas.fill_rects(x=x-2, y=y, width=w*1.2, height=h*1.2)

# styled rects ontop
canvas.fill_styled_rects(x=x, y=y, width=w, height=h, color=rgb)

# outlines
canvas.stroke_style='black'
canvas.stroke_rects(x=x-2, y=y, width=w*1.2, height=h*1.2)

In [ ]:
# batch api arcs
shape = (1000,500)
canvas = Canvas(width=shape[0], height=shape[1], layout=dict(width="100%"))
await canvas.display()

n = 100

# random colors
rgb = np.random.randint(0, 255, size=(n, 3), dtype=np.uint8)

# random positions
x = np.random.randint(0, shape[0], size=n)
y = np.random.randint(0, shape[1], size=n)

# random radii
radius = np.random.randint(10, 50, size=n)

# linear gradient
color_stops = [
    (0,  "rgb(255,255,255)"),
    (1,  "rgb(0,0,0)"),
]
grad = canvas.create_linear_gradient(0, 0, 0,canvas.height,color_stops=color_stops)


# random width and height
start_angle = np.random.uniform(0, 2*pi, size=n)
end_angle = np.random.uniform(0, 2*pi, size=n)


canvas.fill_styled_arcs(x=x, y=y, radius=radius, start_angle=start_angle, end_angle=end_angle, color=rgb)


In [ ]:
# batch api polygons


# helper function to create some polygons
def circle_polygon(x, y, radius, n_points):
    angles = np.linspace(0, 2 * np.pi, n_points, endpoint=False)
    points = np.array([[x + radius * np.cos(angle), y + radius * np.sin(angle)] for angle in angles])
    return points


shape = (1000, 500)
canvas = Canvas(width=shape[0], height=shape[1], layout=dict(width="100%"))
await canvas.display()

# create some polygons
n_polygons = 30
polygons = []
for i in range(n_polygons):

    # circle
    x = np.random.randint(0, shape[0])
    y = np.random.randint(0, shape[1])
    radius = np.random.randint(10, 50)
    n_points = np.random.randint(3, 15)
    polygons.append(circle_polygon(x, y, radius, n_points))

# random colors
rgb = np.random.randint(0, 255, size=(n_polygons, 3), dtype=np.uint8)

canvas.line_width = 10
canvas.fill_polygons(points=[p + 10 for p in polygons])
canvas.stroke_polygons(points=polygons)
# batch api styled polygons
canvas.fill_styled_polygons(points=polygons, color=rgb)

In [ ]:
# batch api for lines

# helper to generate a list of points
# based on a sine wave 
def sine_wave_points(num_points, amplitude=50, frequency=1):
    points = []
    for i in range(num_points):
        x = i * (1000 / num_points)
        y = amplitude * np.sin(frequency * (x / 1000) * (2 * np.pi))
        points.append((x, y))
    return np.array(points, dtype=np.float32)


shape = (1000, 500)
canvas = Canvas(width=shape[0], height=shape[1], layout=dict(width="100%"))
await canvas.display()

n_lines = 10
lines = []
for i in range(n_lines):
    # sine wave
    num_points = np.random.randint(10, 50)
    amplitude = np.random.randint(20, 100)
    frequency = np.random.uniform(0.1, 2.0)
    points = sine_wave_points(num_points, amplitude, frequency)

    # random offset
    offset_x = np.random.randint(0, shape[0])
    offset_y = np.random.randint(0, shape[1])
    points += np.array([offset_x, offset_y])

    lines.append(points)

# random colors
rgb = np.random.randint(0, 255, size=(n_lines, 3), dtype=np.uint8)
canvas.line_width = 10
canvas.stroke_line_segments(points=lines)
canvas.stroke_styled_line_segments(points=lines, color=rgb)


In [ ]:


from collections import deque
canvas = Canvas(width=1000, height=800, layout=dict(width="auto"))
await canvas.display()
import time
import itertools


class Animation(object):

    def __init__(self,canvas):

        self.canvas = canvas
        self.box_pos = (canvas.width / 2, canvas.height / 2)

        self.canvas.on_key_down(self.on_key_down)
        self.canvas.on_mouse_down(self.on_mouse_down)
        self.canvas.on_mouse_wheel(self.on_mouse_wheel)

        self.canvas.on_touch_start(self.on_touch_start)
        self.canvas.on_touch_move(self.on_touch_move)
        self.canvas.on_touch_end(self.on_touch_end)

        self.size = 10
 
    def on_mouse_wheel(self, delta):
        self.size += delta
        self.size = max(1, self.size)
        
    def on_mouse_down(self, x,y):
        self.box_pos = (x,y)


    def on_touch_start(self, x,y):
        print('on_touch_start',x,y)
        self.box_pos = (x,y)
        
    def on_touch_move(self, x,y):
        self.box_pos = (x,y)

    def on_touch_end(self,x,y):
        print("on_touch_end")
    
    
    

    def on_key_down(self, key, shift_key, ctrl_key, alt_key):
        if key == "f":
            self.size +=1
        if key == "g":
            self.size -=1
    

    def __call__(self, dt):
        self.canvas.clear()
        self.canvas.fill_rect(*self.box_pos,self.size,self.size)
       


set_render_loop(canvas, Animation(canvas=canvas))